# List of Commits

Test assignment: https://docs.google.com/document/d/1hnBYR3x798YU5ewLpk8Kz5pEmDnCbQ2wd-lM1ff9wgg/edit#

## 1. Get raw commits

### 1.1. Clone [huggingface/transformers](https://github.com/huggingface/transformers)

In [1]:
!git clone https://github.com/huggingface/transformers.git

### 1.2. Enter repository folder

In [2]:
cd transformers

/home/bernie/Git/List-of-Commits/transformers


### 1.3. Creating files with commits' information

At this step we have access to commits' information. To simlify further process of parsing, I decided to separate commits information into two files: file 'commit_info.csv' with commits' number, number of changed files, number of insertions and number of deletions and file 'commit_message.txt' with commits' messages.

#### Creating 'commit_info.csv'

In [3]:
!echo 'commit_#,changed_files,diff_+,diff_-' > commit_info.csv

In [4]:
!git log --shortstat --pretty='@%H,' | tr "\n" " " | tr "@" "\n" >> commit_info.csv

In [17]:
cat commit_info.csv

commit_#,changed_files,diff_+,diff_-

699e90437f984d69ad3c9b891dd2e9d0fc2cffe4,   1 file changed, 1 insertion(+), 1 deletion(-) 
c54646b13d468b7a21fd6ee18f943ad69daab48e,   3 files changed, 168 insertions(+), 1 deletion(-) 
cc3d0e1b017dbb8dcbba1eb01be77aef7bacee1a,   22 files changed, 1682 insertions(+), 6 deletions(-) 
3a9476d1b412274bcc51143acaaee187e9d18120,   1 file changed, 8 insertions(+), 7 deletions(-) 
60d1f31bb009d09e884699bfe30ac34555bb4a5c,   47 files changed, 89 insertions(+), 89 deletions(-) 
5011efbec81a7a1d094a2eda8bde2b74613ca8b8,   1 file changed, 3 insertions(+), 2 deletions(-) 
504ae9181ca3f0918033f098d10a2c63153e26a6,   1 file changed, 4 insertions(+), 4 deletions(-) 
6cb7d6ec36ec5b3f97f76d8e243cf539fec78949,   1 file changed, 6 insertions(+), 1 deletion(-) 
d752337baabeb8d3d204199938ed8690bee69d70,   1 file changed, 25 insertions(+), 3 deletions(-) 
b67ac44296e920a1df620bb8d79b0754cdf41af8,   1 file changed, 1 insertion(+), 1 deletion(-) 
d51e7c7e8265d69db506828dc

3b20e910b45f392f247468e997b00ce12a4ca5c9,   1 file changed, 2 insertions(+), 6 deletions(-) 
3c12e3c1c45c3acac4bf5f0e1abe7ac64151504c,   1 file changed, 12 insertions(+), 1 deletion(-) 
1f5ea9e04a27171a5034a61999bc6359d19fe4ef,   1 file changed, 1 insertion(+) 
f81077fcf39d858f56cb4404a64c4074a131a5a0,   1 file changed, 1 insertion(+), 1 deletion(-) 
1aed2b908ea21de8e04be206d739e16b290b44c6,   1 file changed, 3 insertions(+) 
a735f727cc93f96abc5b39e776716891667bb565,   1 file changed, 2 insertions(+), 2 deletions(-) 
8c297cdb301336e2d99b9355317e4c3fe0c0eef5,   2 files changed, 36 insertions(+), 14 deletions(-) 
d4d4447d536e5cf8c78518b8b3359168346a4134,   1 file changed, 3 insertions(+), 3 deletions(-) 
7ef40120a0041fed2c43901794cd85b2246455d2,   2 files changed, 103 insertions(+), 14 deletions(-) 
fb2b89840bf2ab9f74702bf83af8ddf92b61efb3,   1 file changed, 15 insertions(+), 22 deletions(-) 
3f48b2bc3e5b555a06492f1e7b999ff29bb6058a,   1 file changed, 1 insertion(+), 1 deletion(-) 
77ffd

9dab39feeab0d141e2353f2b6402e4f823eec2e0,   3 files changed, 35 insertions(+), 3 deletions(-) 
5b193b39b09b30e654d8328f418f632ee618098c,   7 files changed, 132 insertions(+), 46 deletions(-) 
95d1962b9c8460b4cec5a88eb9915e8e25f5bc1e,   1 file changed, 65 insertions(+) 
604a2355dc61f2888d68aab3adb0c5b648a4f42d,   1 file changed, 50 insertions(+) 
77c718edef2a97ab5ffd7fbb80101178e15ae496,   1 file changed, 50 insertions(+) 
325b277db978a609a797c0813d043ead524ee091,   1 file changed, 50 insertions(+) 
d15be2216c2637e51efbce598708e5f9fd8a71d8,   1 file changed, 50 insertions(+) 
f3e23dd90a7dd04c9bd9842944bc4b653332fc49,   1 file changed, 50 insertions(+) 
8b01d15c05ba0c8fcf44550f9a57d5931052ec8b,   1 file changed, 50 insertions(+) 
05bddf304ee2e482cae5a7c06961bf82f317a2e5,   1 file changed, 50 insertions(+) 
783a0c7ee95d2bc1f3e90e448f5a05ae33b7df6c,   1 file changed, 50 insertions(+) 
e7844d60c26837e1dffbd8b89c630f6ae79965f1,   1 file changed, 50 insertions(+) 
b1ee69763c673a6a03dd41bc74e1

#### Creating 'commit_message.txt'

In [5]:
!git log --pretty='%s' > commit_message.txt

In [18]:
cat commit_message.txt

flan-t5.mdx: fix link to large model (#20555)
Add ESM contact prediction (#20535)
[New Model] Add TimeSformer model (#18908)
fix cuda OOM by using single Prior (#20486)
v4.26.0.dev0
Fix link in pipeline device map (#20517)
Fix Hubert models in TFHubertModel and TFHubertForCTC documentation code (#20516)
Fix doctest (#20534)
QnA example: add speed metric (#20522)
update post_process_image_guided_detection (#20521)
Update `ZeroShotObjectDetectionPipeline` doc example (#20528)
add doc for (#20525)
Fix `ConditionalDetrForSegmentation` doc example (#20531)
Fix `PLBart` doctest (#20527)
Change Doctests CI launch time (#20523)
[modelcard] Update dataset tags (#20506)
Add some warning for Dynamo and enable TF32 when it's set (#20515)
Fix Data2VecTextForCasualLM example code documentation (#20510)
Add `natten` for CI (#20511)
Update `AutomaticSpeechRecognitionPipeline` doc example (#20512)
Fix style
Add Chinese-CLIP implementation (#20368)
Fix minimum version for device_map (#20489)
Repurpose t

Initial folder structure for the documentation. A draft of documentation change has been made in the BertModel class.
updates to run_glue
cleaning up run_glue example
clean up glue example
fix model loading
fix python2 tests
tokenization abstract class - tests for examples
remove circle ci parallelism
update readme and setup
Merge pull request #733 from ceremonious/parallel-generation
Merge pull request #620 from chrislarson1/convert-back-to-tf
Merge pull request #745 from leimao/leimao
circle-ci
[BIG] name change
hubs [WIP]
Merge pull request #758 from huggingface/doc
standardizing tokenizers API and adding tests
first commit
parallelism in circlci
adding squad model to xlnet and xlm
Common SequenceSummary class
fix xlm torchscript
Merge branch 'xlnet' of https://github.com/huggingface/pytorch-pretrained-BERT into xlnet
updating tests
WIP XLM + refactoring
Merge pull request #748 from huggingface/torchscript
TorchScript flag in config; Tied weights when not running TorchScript; tuple 

## 2. Parse raw commits

In [6]:
import pandas as pd
import numpy as np

### 2.1. Read 'commit_info.csv' into DataFrame

In [7]:
df = pd.read_csv("commit_info.csv")

In [8]:
df.head()

,commit_#,changed_files,diff_+,diff_-
0,699e90437f984d69ad3c9b891dd2e9d0fc2cffe4,1 file changed,1 insertion(+),1 deletion(-)
1,c54646b13d468b7a21fd6ee18f943ad69daab48e,3 files changed,168 insertions(+),1 deletion(-)
2,cc3d0e1b017dbb8dcbba1eb01be77aef7bacee1a,22 files changed,1682 insertions(+),6 deletions(-)
3,3a9476d1b412274bcc51143acaaee187e9d18120,1 file changed,8 insertions(+),7 deletions(-)
4,60d1f31bb009d09e884699bfe30ac34555bb4a5c,47 files changed,89 insertions(+),89 deletions(-)


The following data processing is made here:
- deleted unnessesary column 'changed_files'
- Nan fields replaces with 0
- in 'diff_+' and 'diff_-' columns numbers are extracted from strings

In [9]:
df.drop("changed_files", axis=1, inplace=True)
df.fillna(0, inplace=True)

In [10]:
df.head()

,commit_#,diff_+,diff_-
0,699e90437f984d69ad3c9b891dd2e9d0fc2cffe4,1 insertion(+),1 deletion(-)
1,c54646b13d468b7a21fd6ee18f943ad69daab48e,168 insertions(+),1 deletion(-)
2,cc3d0e1b017dbb8dcbba1eb01be77aef7bacee1a,1682 insertions(+),6 deletions(-)
3,3a9476d1b412274bcc51143acaaee187e9d18120,8 insertions(+),7 deletions(-)
4,60d1f31bb009d09e884699bfe30ac34555bb4a5c,89 insertions(+),89 deletions(-)


In [11]:
df['diff_+'] = df['diff_+'].str.extract('(\d+)')
df['diff_-'] = df['diff_-'].str.extract('(\d+)')

In [12]:
df.head()

,commit_#,diff_+,diff_-
0,699e90437f984d69ad3c9b891dd2e9d0fc2cffe4,1,1
1,c54646b13d468b7a21fd6ee18f943ad69daab48e,168,1
2,cc3d0e1b017dbb8dcbba1eb01be77aef7bacee1a,1682,6
3,3a9476d1b412274bcc51143acaaee187e9d18120,8,7
4,60d1f31bb009d09e884699bfe30ac34555bb4a5c,89,89


### 2.2. Read 'commit_message.txt' and add 'commit_message' column in DataFrame

In [13]:
f = open("commit_message.txt", "r")
messages = [line.strip() for line in f]

In [14]:
df.insert(1, 'commit_message', messages)

In [15]:
df.head()

,commit_#,commit_message,diff_+,diff_-
0,699e90437f984d69ad3c9b891dd2e9d0fc2cffe4,flan-t5.mdx: fix link to large model (#20555),1,1
1,c54646b13d468b7a21fd6ee18f943ad69daab48e,Add ESM contact prediction (#20535),168,1
2,cc3d0e1b017dbb8dcbba1eb01be77aef7bacee1a,[New Model] Add TimeSformer model (#18908),1682,6
3,3a9476d1b412274bcc51143acaaee187e9d18120,fix cuda OOM by using single Prior (#20486),8,7
4,60d1f31bb009d09e884699bfe30ac34555bb4a5c,v4.26.0.dev0,89,89


### 2.3. Save resulted DataFrame

In [16]:
df.to_csv("list_of_commits.csv")